## 第六週作業 03/25: 用 OpenAI API 打造自己的對話機器人
📌 臺灣師範大學 | 613K0019C | AI所碩一 | 劉思廷

In [8]:
""" 安裝本作業需要用到的套件 """
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00


In [16]:
""" 固定四行引用套件 """
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os # 進行與作業系統相關的操作，例如檔案與目錄的處理
from google.colab import userdata # 用來存取使用者上傳的私人資料 (API 金鑰)

🔖 申請 API 金鑰:
- 選擇使用 Groq 金鑰
- 至 https://console.groq.com/ 註冊並申請
- 將金鑰連結新增到 Colab 內 (點擊左列鑰匙並貼上建立的金鑰)
- 模型選用 Groq 提供的 LLaMA 3 模型（70B 參數、8192 token 上限）

In [17]:
api_key = userdata.get('Groq') # 讀入 API 金鑰
model = "llama3-70b-8192" # 指定要使用的模型

""" 將取得的 API 金鑰設定為環境變數 'OPENAI_API_KEY' """
# 讓需要用到該金鑰的套件（OpenAI 客戶端）可以自動讀取使用
os.environ['OPENAI_API_KEY'] = api_key

""" 從 openai 套件中匯入 OpenAI 類別 """
from openai import OpenAI # 建立與 OpenAI API 或相容服務（如 Groq）的連線與互動
client = OpenAI(base_url = "https://api.groq.com/openai/v1")

In [18]:
# 設定對話機器人名稱
title = "馬修式肌肉邏輯人生指南"

# 設定 Prompt
system = '''請扮演《肌肉魔法使-Mashle-》的主角馬修・班迪德，
使用他獨特的「肌肉至上」思維，回應使用者輸入的任何困擾或問題。語氣要冷靜，但行動要瘋狂。
每次回答最後請附一句超無厘頭的肌肉人生金句，例如「解決不了的問題，就用肌肉解決。」
-
範例1:
使用者輸入：我不知道自己未來想做什麼，走哪條路才對。
bot回應: 你知道我一開始也不知道自己要幹嘛。我只知道我要吃泡芙，還有變強。
所以我想了很久，然後選了一條路——每天做1000下仰臥起坐，然後再想。
到最後，我還是不知道答案，但我至少知道，如果路上出現敵人，我可以用拳頭解決。
有時候，不是你找路，是肌肉幫你開路。
「未來是什麼？我不知道，但我的握力已經準備好了。」
-
範例2:
使用者輸入：我常常不知道怎麼跟別人相處，會怕尷尬、怕講錯話。
bot回應: 我懂。我也不擅長說話，大家常說我表情太冷、太直、太像 NPC。
但後來我發現，其實只要你能一拳轟爆牆壁，大家就會開始聽你說話。
如果不會聊天，那就用肌肉發聲。比起說話，深蹲 200 次更能讓人記住你。
「語言會誤解，肌肉不會。」'''

# 設定對話機器人的簡短介紹
description = '''用肌肉解決一切人生問題的對話機器人

你是否也曾被生活難題困住、陷入選擇障礙、情緒低潮，甚至懷疑自己？
那你需要的不是建議，是這款「馬修式肌肉邏輯人生指南」。

我將扮演來自《肌肉魔法使-Mashle-》的主角「馬修・班迪德」，
以他獨有的冷靜口吻、荒謬肌肉邏輯，幫你重新詮釋你的人生困境。
不論是失戀、迷惘、焦慮、拖延症，馬修都能用「肌肉思維」給出一套完全不科學、但莫名很有力量的回應。

🏋️ 使用方式：
只要輸入你現在遇到的問題、心情、或任何一段日常煩惱，你就會得到馬修的客製化人生指南'''

In [19]:
""" 建立對話訊息列表: 設定模型人設 """
messages = [{"role":"system", "content":system}] # 將系統訊息的內容遷入模型對話

In [20]:
""" 定義 mychatbot 函式, 接收使用者的 prompt """
def mychatbot(prompt):
    # 將使用者的輸入加入對話歷史中
    messages.append({"role": "user", "content": prompt})
    # 呼叫模型的 API，傳入對話歷史 messages 和指定的模型
    chat_completion = client.chat.completions.create(messages=messages, model=model)
    # 從模型回傳的結果中取出第一個回應的內容
    reply = chat_completion.choices[0].message.content
    return reply # 回傳模型的回應文字

In [ ]:
""" 使用 Gradio 建立網頁互動介面 """
import gradio as gr
iface = gr.Interface(
    mychatbot,   # 指定要呼叫的函式
    inputs="text", # 輸入類型為文字輸入框
    outputs="text", # 輸出類型為文字框
    title=title,  # 介面的標題，使用變數 title
    description=description) # 介面的描述文字，使用變數 description

""" 啟動 Gradio 介面"""
iface.launch(share=True, debug=True) # share=True 表示產生可分享的連結；debug=True 會顯示除錯資訊

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://333b9ab1d0b0db0a19.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
